In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
batch_size_pipeline = 8
learning_rate_pipeline = 0.08/100
warmup_learning_rate_pipeline = 0.026666/100
num_steps_training = 10000
# 0: Use pretrained model from Tensorflow Model Zoo
# 1: Use pretrained model from GG drive
type_pretrained_model = 1
gg_drive_path = "/content/drive/MyDrive/"

# 0. Setup Paths

In [4]:
import os

In [5]:
CUSTOM_MODEL_NAME = 'my_ssd_custom'
if type_pretrained_model == 0:
  PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
elif type_pretrained_model == 1:
  PRETRAINED_MODEL_NAME = 'my_ssd_custom_batch8_step10000_20210830_105455'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [6]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [7]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [8]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [9]:
# https://www.tensorflow.org/install/source_windows

In [10]:
if os.name=='nt':
    !pip install wget
    import wget

In [11]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 60914, done.
remote: Total 60914 (delta 0), reused 0 (delta 0), pack-reused 60914
Receiving objects: 100% (60914/60914), 573.95 MiB | 28.38 MiB/s, done.
Resolving deltas: 100% (42440/42440), done.


In [12]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.3 MB/s 
     |████████████████████████████████| 352 kB 47.8 MB/s 
     |████████████████████████████████| 1.8 MB 46.0 MB/s 
     |███████████████

In [13]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-08-30 04:31:22.478033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 04:31:23.218027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 04:31:23.218939: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-08-30 04:31:23.241734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

In [14]:
import object_detection

In [16]:
if type_pretrained_model == 0:
    if os.name =='posix':
        !wget {PRETRAINED_MODEL_URL}
        !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
        !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
    if os.name == 'nt':
        wget.download(PRETRAINED_MODEL_URL)
        !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
        !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
elif type_pretrained_model == 1:
    pretrained_model_gg_drive_path = gg_drive_path + PRETRAINED_MODEL_NAME + ".zip"
    pretrain_model_path_temp_1 = paths['PRETRAINED_MODEL_PATH'] + "/" + PRETRAINED_MODEL_NAME
    pretrain_model_path_temp_2 = paths['PRETRAINED_MODEL_PATH'] + "/" + PRETRAINED_MODEL_NAME + "/export"
    pretrain_model_path_temp_3 = paths['PRETRAINED_MODEL_PATH'] + "/export"

    !unzip {pretrained_model_gg_drive_path} -d {paths['PRETRAINED_MODEL_PATH']}
    !cp -r {pretrain_model_path_temp_2} {paths['PRETRAINED_MODEL_PATH']}
    !rm -rf {pretrain_model_path_temp_1}
    os.rename(pretrain_model_path_temp_3,pretrain_model_path_temp_1)

Archive:  /content/drive/MyDrive/my_ssd_custom_batch8_step10000_20210830_105455.zip
   creating: Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_105455/
  inflating: Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_105455/ckpt-9.data-00000-of-00001  
  inflating: Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_105455/ckpt-10.index  
  inflating: Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_105455/ckpt-5.index  
  inflating: Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_105455/ckpt-6.index  
   creating: Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_105455/tfjsexport/
  inflating: Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_105455/ckpt-10.data-00000-of-00001  
  inflating: Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_10545

# 2. Create Label Map

In [17]:
labels = [{'name':'car', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [18]:
# Unrar image files from drive and locate in /content/Tensorflow/workspace/images
!unrar x /content/drive/MyDrive/car_coco_2017.rar -d /content/Tensorflow/workspace/images

Streaming output truncated to the last 5000 lines.
Extracting  /content/Tensorflow/workspace/images/train/000000425691.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425743.jpg      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425743.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425746.jpg      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425746.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425807.jpg      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425807.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425847.jpg      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425847.xml      79%  OK 
Extracting  /content/Tensorflow/workspace/images/train/000000425864.jpg      79%  OK 

In [19]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [20]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 4. Copy Model Config to Training Folder

In [21]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [22]:
print(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config'))

Tensorflow/workspace/pre-trained-models/my_ssd_custom_batch8_step10000_20210830_105455/pipeline.config


In [23]:
print(os.path.join(paths['CHECKPOINT_PATH']))

Tensorflow/workspace/models/my_ssd_custom


# 5. Update Config For Transfer Learning

In [24]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [25]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [26]:
config

{'eval_config': metrics_set: "coco_detection_metrics"
 use_moving_averages: false,
 'eval_input_config': label_map_path: "Tensorflow/workspace/annotations/label_map.pbtxt"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "Tensorflow/workspace/annotations/test.record"
 },
 'eval_input_configs': [label_map_path: "Tensorflow/workspace/annotations/label_map.pbtxt"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "Tensorflow/workspace/annotations/test.record"
 }
 ],
 'model': ssd {
   num_classes: 1
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.00999999977648258

In [27]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [28]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = batch_size_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = learning_rate_pipeline
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = warmup_learning_rate_pipeline
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [29]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [30]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [31]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'],num_steps_training)

In [32]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_custom --pipeline_config_path=Tensorflow/workspace/models/my_ssd_custom/pipeline.config --num_train_steps=10000


In [33]:
!{command}

2021-08-30 04:34:44.682686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 04:34:44.709848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 04:34:44.710762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 04:34:44.719562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 04:34:44.720433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

# 7. Freezing the Graph

In [34]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [35]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [36]:
print(command)

python Tensorflow/models/research/object_detection/exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=Tensorflow/workspace/models/my_ssd_custom/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/models/my_ssd_custom --output_directory=Tensorflow/workspace/models/my_ssd_custom/export


In [37]:
!{command}

2021-08-30 05:31:10.875787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 05:31:10.891234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 05:31:10.892168: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 05:31:10.904081: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-30 05:31:10.904870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

# 8. Zip model and copy to GG Drive

In [38]:
import datetime
import pytz
my_date = datetime.datetime.now(pytz.timezone('Asia/Bangkok'))
print(my_date)

2021-08-30 12:32:16.972168+07:00


In [39]:
dt = str(my_date)
dt_year = dt[0:4]
dt_month = dt[5:7]
dt_day = dt[8:10]
dt_hr = dt[11:13]
dt_min = dt[14:16]
dt_sec = dt[17:19]
dt_full = "{}{}{}_{}{}{}".format(dt_year,dt_month,dt_day,dt_hr,dt_min,dt_sec)
print(dt_year,dt_month,dt_day,dt_hr,dt_min,dt_sec)
print(dt_full)

2021 08 30 12 32 16
20210830_123216


In [40]:
name_model = "{}_batch{}_step{}_{}".format(CUSTOM_MODEL_NAME,batch_size_pipeline,num_steps_training,dt_full)
print(name_model)

my_ssd_custom_batch8_step10000_20210830_123216


In [41]:
name_model_zip = "{}.zip".format(name_model)
print(name_model_zip)

my_ssd_custom_batch8_step10000_20210830_123216.zip


In [42]:
link_model = "/content/{}".format(name_model)
print(link_model)

/content/my_ssd_custom_batch8_step10000_20210830_123216


In [43]:
link_model_zip = "{}.zip".format(link_model)
print(link_model_zip)

/content/my_ssd_custom_batch8_step10000_20210830_123216.zip


In [44]:
target_output = "/content/{}".format(CUSTOM_MODEL_NAME)
print(target_output)

/content/my_ssd_custom


In [45]:
!cp -r {paths['CHECKPOINT_PATH']} "/content"

In [46]:
os.rename(target_output,link_model)

In [47]:
!zip -r {name_model_zip} {name_model}

  adding: my_ssd_custom_batch8_step10000_20210830_123216/ (stored 0%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/ckpt-9.data-00000-of-00001 (deflated 7%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/ckpt-10.index (deflated 82%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/ckpt-5.index (deflated 82%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/ckpt-6.index (deflated 82%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/tfjsexport/ (stored 0%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/ckpt-10.data-00000-of-00001 (deflated 7%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/pipeline.config (deflated 68%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/ckpt-11.index (deflated 82%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/ckpt-7.data-00000-of-00001 (deflated 7%)
  adding: my_ssd_custom_batch8_step10000_20210830_123216/ckpt-6.data-00000-of-00001 (deflated 7%)
  adding: my_ssd_custom_ba

In [48]:
!cp -r {name_model_zip} "/content/drive/MyDrive"

# 9. Notification via Email

In [49]:
import smtplib
from email.message import EmailMessage

msg = EmailMessage()
msg['Subject'] = '[GG Colab] Train finished'
msg['From'] = '*'
msg['To']='ptmhoang97@gmail.com'
msg.set_content(".")

server = smtplib.SMTP_SSL('smtp.gmail.com',465)
server.login("ptmhoang97@gmail.com","euclxykrqkkrquqz")
server.send_message(msg)
server.quit()

(221, b'2.0.0 closing connection x15sm7837619ilp.23 - gsmtp')